In [1]:
%matplotlib inline

In [2]:
import keras

Using TensorFlow backend.


In [34]:
from math import ceil

In [45]:
from datetime import datetime

In [3]:
DATA_PATH = 'data/'

In [43]:
MODEL_PATH = 'models/'

In [4]:
BATCH_SIZE = 64

## Load data

In [5]:
from keras.applications.xception import Xception

In [6]:
from keras.preprocessing import image

In [8]:
from keras.models import Model

In [9]:
from keras.layers import Dense, GlobalAveragePooling2D

In [29]:
from keras.optimizers import Adam

In [13]:
train_generator = image.ImageDataGenerator()

In [15]:
train_batches = train_generator.flow_from_directory(
    DATA_PATH + 'train/',
    target_size=(299, 299),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 12929 images belonging to 49 classes.


In [16]:
valid_generator = image.ImageDataGenerator()

In [17]:
valid_batches = valid_generator.flow_from_directory(
    DATA_PATH + 'valid/',
    target_size=(299, 299),
    shuffle=True,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
)

Found 3256 images belonging to 49 classes.


In [20]:
num_classes = train_batches.num_classes

In [11]:
base_model = Xception(weights='imagenet', include_top=False)

83689472/83683744 [==============================] - 5s 0us/step


In [24]:
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(num_classes, activation='softmax')(x)

In [25]:
model = Model(inputs=base_model.input, outputs=predictions)

In [27]:
for layer in base_model.layers:
    layer.trainable = False

In [32]:
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

In [31]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, None, None, 3 128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, None, None, 3 0           block1_conv1_bn[0][0]            
__________________________________________________________________________________________________
block1_con

In [36]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=1,
)

Epoch 1/1
203/203 [==============================] - 308s 2s/step - loss: 2.7868 - acc: 0.2393 - val_loss: 2.4143 - val_acc: 0.3127


In [38]:
model.optimizer.lr = 0.01

In [39]:
model.fit_generator(
    train_batches,
    steps_per_epoch=int(ceil(train_batches.samples/BATCH_SIZE)),
    validation_data=valid_batches,
    validation_steps=int(ceil(valid_batches.samples/BATCH_SIZE)),
    epochs=8,
)

Epoch 1/8
203/203 [==============================] - 304s 1s/step - loss: 2.1944 - acc: 0.3817 - val_loss: 2.1443 - val_acc: 0.3796
Epoch 2/8
203/203 [==============================] - 307s 2s/step - loss: 1.9402 - acc: 0.4469 - val_loss: 2.0021 - val_acc: 0.4235
Epoch 3/8
203/203 [==============================] - 306s 2s/step - loss: 1.7665 - acc: 0.4988 - val_loss: 1.8995 - val_acc: 0.4487
Epoch 4/8
203/203 [==============================] - 306s 2s/step - loss: 1.6738 - acc: 0.5264 - val_loss: 1.8462 - val_acc: 0.4656
Epoch 5/8
203/203 [==============================] - 306s 2s/step - loss: 1.5581 - acc: 0.5553 - val_loss: 1.9765 - val_acc: 0.4125
Epoch 6/8
203/203 [==============================] - 305s 2s/step - loss: 1.4874 - acc: 0.5729 - val_loss: 1.7510 - val_acc: 0.4886
Epoch 7/8
203/203 [==============================] - 306s 2s/step - loss: 1.4311 - acc: 0.5985 - val_loss: 1.7070 - val_acc: 0.4957
Epoch 8/8
203/203 [==============================] - 305s 2s/step - loss: 1.

In [55]:
model_name = MODEL_PATH + f'carnet-{datetime.now().isoformat()}'

model.save_weights(model_name + '.h5')
with open(model_name + '.json', 'w') as f:
    f.write(model.to_json())